In [ ]:
import time

csv_path = "data/sprint2/PASS_ALL_202503242210.csv" # путь к жирному файлу

parquet_path = csv_path.replace('.csv', '.parquet')
print(f"init_completed {time.ctime()}")


In [ ]:
import sys
!{sys.executable} -m pip install pyarrow duckdb polars plotly jupyter_dash dash --upgrade
# зависимости


import duckdb
import polars as pl
import plotly
import jupyter_dash

print("DuckDB OK:", duckdb.__version__)
print("Polars OK:", pl.__version__)
print("Plotly OK:", plotly.__version__)
print("Jupiter_dash OK:", jupyter_dash.__version__)


In [ ]:
import polars as pl
import os
from pathlib import Path


def csv_to_parquet(csv_path: str, columns: list[str] = None, sep: str = ";") -> str:
    """
    Преобразует CSV-файл в Parquet, оставляя только нужные колонки.

    Аргументы:
    - csv_path: путь к исходному CSV
    - columns: список нужных колонок (или None, чтобы взять все)
    - sep: разделитель (по умолчанию ';')

    Возвращает путь к сохранённому Parquet-файлу.
    """
    csv_path = Path(csv_path)
    if not csv_path.exists():
        raise FileNotFoundError(f"Файл {csv_path} не найден.")

    print(f"📥 Чтение файла {csv_path.name}...")
    df = pl.read_csv(
        csv_path,
        separator=sep,
        columns=columns,
    )

    parquet_path = csv_path.with_suffix(".parquet")

    print(f"💾 Сохранение в {parquet_path.name}...")
    df.write_parquet(parquet_path)

    print(f"✅ Готово: {parquet_path} ({df.shape[0]} строк, {df.shape[1]} колонок)")
    return str(parquet_path)

if not os.path.exists(parquet_path):
    csv_to_parquet(csv_path, columns=["TRAN_DATE",
                                      "DEVICE_NO",
                                      "CRD_NO",
                                      "TRANSFER_TYPE_ID",
                                      "TRANSPORT_TYPE_ID",
                                      "GD_ID",
                                      "PLACE_ID",
                                      "BUS_RT_NO"])



In [ ]:
# import duckdb
# import polars as pl
#
# # Выбор нужных колонок через DuckDB и сразу -> Polars
# query = f"""
#     SELECT
#         TRAN_DATE,
#         DEVICE_NO,
#         CRD_NO,
#         TRANSFER_TYPE_ID,
#         TRANSPORT_TYPE_ID,
#         GD_ID,
#         PLACE_ID
#     FROM read_csv_auto('{csv_path}', delim=';')
# """
#
# df = duckdb.query(query).pl()
# df.head()


In [ ]:
import duckdb
import polars as pl


def read_parquet_with_duckdb(data_path: str):
    """
    Читает Parquet файл с использованием DuckDB и возвращает результат в формате Polars DataFrame.

    Аргументы:
    - parquet_path: путь к Parquet файлу.

    Возвращает Polars DataFrame.
    """
    # Подключаемся к DuckDB
    conn = duckdb.connect()

    # Запрос для чтения Parquet файла
    query = f"""SELECT
                TRAN_DATE,
                DEVICE_NO,
                CRD_NO,
                TRANSFER_TYPE_ID,
                TRANSPORT_TYPE_ID,
                GD_ID,
                PLACE_ID,
                BUS_RT_NO
            FROM '{data_path}' WHERE TRANSPORT_TYPE_ID>1"""

    return duckdb.query(query).pl()


#Делаем полезные преобразования, чтобы дальше проще работать
df = read_parquet_with_duckdb(parquet_path)
df = df.with_columns(
    pl.col("TRAN_DATE").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S%.f")
)
df = df.with_columns(
    pl.when(
        (pl.col("TRANSPORT_TYPE_ID") == 1) & (pl.col("BUS_RT_NO").is_null())
    )
    .then(-239)
    .otherwise(pl.col("BUS_RT_NO"))
    .alias("BUS_RT_NO")
)

df = df.with_columns(
    pl.col("TRAN_DATE").dt.strftime("%A").alias("DAY_NAME")
)

df.head()


In [ ]:
def show(fig):
    """
    Настройки цветов и всякой красоты для pyplot.express
    :param fig: какой график украшаем
    :return:
    """
    fig.update_layout(
        updatemenus=[{
            "type": "buttons",
            "showactive": False,
            "buttons": [{
                "label": "Play",
                "method": "animate",
                "args": [None, {
                    "frame": {"duration": 2000, "redraw": True},  # 1000 мс = 1 секунда на кадр
                    "fromcurrent": True,
                    "transition": {"duration": 500, "easing": "linear"}  # плавность
                }]
            }, {
                "label": "Pause",
                "method": "animate",
                "args": [[None], {
                    "frame": {"duration": 0, "redraw": False},
                    "mode": "immediate",
                    "transition": {"duration": 0}
                }]
            }]
        }],
        sliders=[{
            "transition": {"duration": 500},
            "currentvalue": {"prefix": "День: "},
            "pad": {"t": 30},
            "len": 0.9
        }]
    )

    fig.show()

In [ ]:
import polars as pl
import matplotlib.pyplot as plt
import pandas
import plotly.express as px
from datetime import datetime

default_top = 20
start = datetime.strptime("2025-03-10 05:00:00", "%Y-%m-%d %H:%M:%S")
end = datetime.strptime("2025-03-11 23:59:00", "%Y-%m-%d %H:%M:%S")

def get_top_routes_and_stops(
        df: pl.DataFrame,
        start_time: pl.datetime,
        end_time: pl.datetime,
        top_n: int = default_top
):
    """
    Анализирует популярные маршруты и остановки за заданный временной интервал.

    Аргументы:
    - df: Polars DataFrame с данными.
    - start_time: строка в формате "YYYY-MM-DD HH:MM:SS"
    - end_time: строка в том же формате
    - top_n: сколько маршрутов/остановок показать в топе.

    Возвращает два датафрейма: топ маршрутов и топ остановок.
    """

    route_lookup = pl.read_csv("data/sprint2/REF_TRANSPORT_WAY_202503251803.csv", separator=';')
    place_lookup = pl.read_csv("data/sprint2/REF_PSG_PLACES_202503251822.csv", separator=';')
    # Фильтрация по времени
    filtered = df.filter(
        (pl.col("TRAN_DATE") >= start_time) &
        (pl.col("TRAN_DATE") <= end_time) & (pl.col("TRANSPORT_TYPE_ID") != 1)
    )
    # print(filtered.head())
    # Топ маршрутов
    top_routes = (
        filtered.join(
            route_lookup.select(["WAY_ID", "NAME", "TRANSPORT_ID"]),
            left_on="BUS_RT_NO",
            right_on="WAY_ID",
            how="left").filter(pl.col("BUS_RT_NO").is_not_null())
        .group_by(["BUS_RT_NO", "NAME", "TRANSPORT_ID", "DAY_NAME"])
        .agg([pl.len().alias("count"),
              pl.col("DEVICE_NO").n_unique().alias("vehicle_count")
              ])
        .sort("DAY_NAME", "count", descending=[False, True])
        .drop_nulls()
        .with_columns([
            (pl.col("count") / pl.col("vehicle_count")).alias("rides_per_vehicle")
        ])
        .with_columns([
            pl.col("count").rank("dense", descending=True).over("DAY_NAME").alias("popularity"),
            pl.col("rides_per_vehicle").rank("dense", descending=True).over("DAY_NAME").alias("overload")]
        )
        .filter((pl.col("popularity") <= 20) | (pl.col("overload") <= 20))
    )

    # Топ остановок
    top_stops = (
        filtered.join(
            place_lookup.select(["PLACE_ID", "NAME", "LN_NAME_SHORT"]),
            left_on="PLACE_ID",
            right_on="PLACE_ID",
            how="left"
        )
        .group_by("PLACE_ID", "NAME", "LN_NAME_SHORT", "DAY_NAME")
        .agg(pl.len().alias("count"))
        .sort("DAY_NAME", "count", descending=[False, True])
        .drop_nulls()
        .with_columns(
            pl.col("count").rank("dense", descending=True).over("DAY_NAME").alias("rank")
        )
        .filter(pl.col("rank") <= 20))

    return top_routes, top_stops


def plot_top_routes(top: pl.DataFrame, start_time: pl.datetime, end_time: pl.datetime, top_n: int = default_top, do_show=True):
    df_pandas = top.to_pandas()
    day_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    df_pandas["DAY_NAME"] = pandas.Categorical(df_pandas["DAY_NAME"], categories=day_order, ordered=True)

    df_pandas = df_pandas.sort_values("DAY_NAME")

    fig = px.bar(
        df_pandas,
        x="NAME",
        y="count",
        color="TRANSPORT_ID",  # если хочешь по видам транспорта
        title=f"Топ-{top_n} маршрутов с {start_time} по {end_time}",
        animation_frame="DAY_NAME", animation_group="NAME"
    )
    fig.update_layout(xaxis_categoryorder="total descending")

    fig.update_layout(
        xaxis_title="Маршрут",
        yaxis_title="Количество поездок",
        xaxis_tickangle=-45,
        height=500,
        width=1000,
        margin=dict(t=50, b=100),
    )
    if do_show:
        show(fig)
    return fig

def plot_top(top: pl.DataFrame, color: str, y: str = "count", start_time: pl.datetime = start,
             end_time: pl.datetime = end, top_n: int = default_top, ):
    """
    я попытался обобщить постройку графиков, но не получилось
    :param top: даные
    :param color: название столбца, в котором группировка по цветам
    :param y: данные по Y
    :param start_time: от какого времени
    :param end_time: до какого времени
    :param top_n: топ сколько выводить
    :return:
    """
    df_pandas = top.to_pandas()
    day_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    df_pandas["DAY_NAME"] = pandas.Categorical(df_pandas["DAY_NAME"], categories=day_order, ordered=True)

    df_pandas = df_pandas.sort_values("DAY_NAME")

    fig = px.bar(
        df_pandas,
        x="NAME",
        y=y,
        color=color,
        title=f"Топ-{top_n} остановок с {start_time} по {end_time}",
        animation_frame="DAY_NAME", animation_group="NAME",
    )
    fig.update_layout(xaxis_categoryorder="total descending")

    fig.update_layout(
        xaxis_title="Остановка",
        yaxis_title="Количество поездок",
        xaxis_tickangle=-45,
        height=500,
        width=1000,
        margin=dict(t=50, b=100),
    )
    show(fig)

In [ ]:
from datetime import datetime

start = datetime.strptime("2025-03-10 05:00:00", "%Y-%m-%d %H:%M:%S")
end = datetime.strptime("2025-03-16 23:59:00", "%Y-%m-%d %H:%M:%S")

top_routes, top_stops = get_top_routes_and_stops(df, start, end)
#ниже этой функции ничего полезного нет - дальше можно уже играться

plot_top(top_stops, color="LN_NAME_SHORT")


In [ ]:
plot_top(top_routes.filter(pl.col("popularity") <= 20), color="TRANSPORT_ID")

plot_top(top_routes.filter(pl.col("overload") <= 20), color="TRANSPORT_ID", y="rides_per_vehicle")


In [ ]:
start = datetime.strptime("2025-03-12 06:00:00", "%Y-%m-%d %H:%M:%S")
end = datetime.strptime("2025-03-13 06:00:00", "%Y-%m-%d %H:%M:%S")
top_routes, top_stops = get_top_routes_and_stops(df, start, end)
plot_top_routes(top_routes, start, end)

start = datetime.strptime("2025-03-16 06:00:00", "%Y-%m-%d %H:%M:%S")
end = datetime.strptime("2025-03-17 06:00:00", "%Y-%m-%d %H:%M:%S")
top_routes, top_stops = get_top_routes_and_stops(df, start, end)

plot_top_routes(top_routes, start, end)

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import pandas as pd
from jupyter_dash import JupyterDash

fff = None
def application(df):
    app = Dash(__name__)
    app.layout = html.Div([
        dcc.DatePickerRange(
            id='date-picker',
            min_date_allowed=df['TRAN_DATE'].min().date(),
            max_date_allowed=df['TRAN_DATE'].max().date(),
            start_date=df['TRAN_DATE'].min().date(),
            end_date=df['TRAN_DATE'].max().date()
        ),
        dcc.Graph(id='graph')
    ])

    @app.callback(
        Output('graph', 'figure'),
        Input('date-picker', 'start_date'),
        Input('date-picker', 'end_date')
    )
    def update_graph(start_date, end_date):
        start_date = pd.to_datetime(start_date)
        end_date = pd.to_datetime(end_date)
        t1, t2 = get_top_routes_and_stops(df, start_date, end_date)
        fig = plot_top_routes(t1, start_date, end_date, do_show=False)
        return fig

    app.run(mode='inline', port = 8239)



In [ ]:
application(df)

In [ ]:
fff.show()